In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn import metrics

In [2]:
data = pd.read_csv('data/data_DFT.csv')
y = pd.DataFrame(data['Yield'],columns=['Yield'])
X = data.drop(columns=['Name', 'ID', 'Yield'])
    
data_s = pd.read_csv(f'data/data_DFT_ALL.csv')
y_s = pd.DataFrame(data_s['Yield'],columns=['Yield'])
X_s = data_s.drop(columns=['Name', 'ID', 'Yield'])


r2_train = []
r2_test = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=i)
    param = {"n_estimators": [100, 500, 1000], "gamma": [0, 1, 2], "min_child_weight": [1, 2, 5, 10],
             "subsample": [0.6, 1], "eta": [0.01, 0.1, 0.3, 1], "colsample_bytree": [0.6, 1],
             "lambda": [0.01, 0.1, 1], "max_depth": [5, 7, 9]}
    reg = RandomizedSearchCV(XGBRegressor(random_state=0), param_distributions=param, n_iter=600, cv=5, random_state=0, n_jobs=32)
    reg.fit(X_s, y_s['Yield'])
    best = reg.best_estimator_
    y_pred1 = best.predict(X_s)
    y_pred2 = best.predict(X_test)
    r2_train.append(metrics.r2_score(y_s, y_pred1))
    r2_test.append(metrics.r2_score(y_test, y_pred2))

r2_train = pd.DataFrame(data=r2_train, columns=['r2_train'])
r2_test = pd.DataFrame(data=r2_test, columns=['r2_test'])
result = pd.concat([r2_train, r2_test], axis=1, join='inner')
result.to_csv(f'result/result_DFT_wotarget.csv', index=False)
print('Avg_R2_test:', r2_test['r2_test'].mean())
print('Max_R2_test:', r2_test['r2_test'].max())
print('Std_R2_test:', r2_test['r2_test'].std(ddof=0))

Avg_R2_test: 0.16215976205638621
Max_R2_test: 0.6118234796879065
Std_R2_test: 0.21867515129417853
